# 4. 고급 기법 및 최적화

이 노트북에서는 유전 알고리즘의 성능을 향상시키는 다양한 고급 기법들을 학습합니다.

## 목차
1. [다양한 선택 방법 비교](#1-다양한-선택-방법-비교)
2. [교차 전략 비교](#2-교차-전략-비교)
3. [적응적 돌연변이](#3-적응적-돌연변이)
4. [엘리트 전략과 다양성 보존](#4-엘리트-전략과-다양성-보존)
5. [하이퍼파라미터 최적화](#5-하이퍼파라미터-최적화)

---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from typing import List, Tuple, Callable
import sys
sys.path.append('..')

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (15, 8)

print("고급 기법 노트북 준비 완료!")


## 1. 다양한 선택 방법 비교

유전 알고리즘에서 부모 선택은 매우 중요합니다. 다양한 선택 방법을 구현하고 비교해보겠습니다.

### 선택 방법들:
1. **룰렛 휠 선택**: 적합도에 비례한 확률로 선택
2. **토너먼트 선택**: 무작위 그룹에서 최고 개체 선택  
3. **랭킹 선택**: 적합도 순위 기반 선택
4. **엘리트 선택**: 상위 개체들만 선택


In [ ]:
class SelectionMethods:
    """다양한 선택 방법들을 구현한 클래스"""
    
    @staticmethod
    def roulette_wheel_selection(population, fitness_scores, num_parents):
        """룰렛 휠 선택"""
        # 음수 적합도 처리
        min_fitness = np.min(fitness_scores)
        if min_fitness < 0:
            adjusted_fitness = fitness_scores - min_fitness + 1
        else:
            adjusted_fitness = fitness_scores
        
        # 확률 계산
        total_fitness = np.sum(adjusted_fitness)
        probabilities = adjusted_fitness / total_fitness
        
        # 누적 확률
        cumulative_prob = np.cumsum(probabilities)
        
        parents = []
        for _ in range(num_parents):
            r = random.random()
            selected_idx = np.searchsorted(cumulative_prob, r)
            parents.append(population[selected_idx].copy())
        
        return parents
    
    @staticmethod
    def tournament_selection(population, fitness_scores, num_parents, tournament_size=3):
        """토너먼트 선택"""
        parents = []
        for _ in range(num_parents):
            # 토너먼트 참가자 선택
            candidates = random.sample(range(len(population)), tournament_size)
            winner_idx = max(candidates, key=lambda x: fitness_scores[x])
            parents.append(population[winner_idx].copy())
        
        return parents
    
    @staticmethod
    def rank_selection(population, fitness_scores, num_parents):
        """랭킹 선택"""
        # 적합도 순으로 정렬
        sorted_indices = np.argsort(fitness_scores)
        
        # 랭킹 기반 확률 (선형)
        ranks = np.arange(1, len(population) + 1)
        probabilities = ranks / np.sum(ranks)
        
        # 누적 확률
        cumulative_prob = np.cumsum(probabilities)
        
        parents = []
        for _ in range(num_parents):
            r = random.random()
            selected_rank = np.searchsorted(cumulative_prob, r)
            selected_idx = sorted_indices[selected_rank]
            parents.append(population[selected_idx].copy())
        
        return parents
    
    @staticmethod
    def elite_selection(population, fitness_scores, num_parents):
        """엘리트 선택"""
        # 상위 개체들만 선택
        elite_indices = np.argsort(fitness_scores)[-num_parents:]
        parents = [population[idx].copy() for idx in elite_indices]
        return parents

# 선택 방법별 효과 테스트
def test_selection_methods():
    """선택 방법들의 효과를 테스트"""
    
    # 가상의 개체군과 적합도 생성
    population_size = 20
    population = [np.random.randn(10) for _ in range(population_size)]
    fitness_scores = np.random.exponential(2, population_size)  # 편향된 적합도 분포
    
    # 각 선택 방법 테스트
    selection_methods = {
        'Roulette Wheel': SelectionMethods.roulette_wheel_selection,
        'Tournament': SelectionMethods.tournament_selection,
        'Rank': SelectionMethods.rank_selection,
        'Elite': SelectionMethods.elite_selection
    }
    
    plt.figure(figsize=(15, 10))
    
    # 원본 적합도 분포
    plt.subplot(2, 3, 1)
    plt.hist(fitness_scores, bins=10, alpha=0.7, color='gray')
    plt.title('Original Fitness Distribution')
    plt.xlabel('Fitness')
    plt.ylabel('Count')
    
    # 각 선택 방법별 결과
    for i, (name, method) in enumerate(selection_methods.items(), 2):
        selected_parents = method(population, fitness_scores, 10)
        
        # 선택된 부모들의 적합도
        selected_indices = []
        for parent in selected_parents:
            # 가장 유사한 개체 찾기 (단순화)
            distances = [np.linalg.norm(parent - ind) for ind in population]
            selected_indices.append(np.argmin(distances))
        
        selected_fitness = [fitness_scores[idx] for idx in selected_indices]
        
        plt.subplot(2, 3, i)
        plt.hist(selected_fitness, bins=8, alpha=0.7)
        plt.title(f'{name} Selection')
        plt.xlabel('Fitness')
        plt.ylabel('Count')
        
        print(f"{name} - 평균 적합도: {np.mean(selected_fitness):.3f}, 표준편차: {np.std(selected_fitness):.3f}")
    
    plt.tight_layout()
    plt.show()

test_selection_methods()
